In [1]:
import pandas as pd 
import sys
import os 
from dotenv import load_dotenv


sys.path.append('C:\\Users\\Ourba\\Desktop\\10Academy\\10ac_week1_telecom_analysis\\src')
from loader import load_data


load_dotenv()

user = os.getenv('PG_USER')
password = os.getenv('PG_PASSWORD')
database_name = 'telecom'
table_name= 'xdr_data'
host = 'localhost'
port = '5432'

#Load data from Loader Module
df = load_data(host, port, user, password, database_name, table_name)
print(df.head())

      Bearer Id            Start  Start ms              End  End ms  \
0  1.311448e+19   4/4/2019 12:01     770.0  4/25/2019 14:35   662.0   
1  1.311448e+19   4/9/2019 13:04     235.0   4/25/2019 8:15   606.0   
2  1.311448e+19   4/9/2019 17:42       1.0  4/25/2019 11:58   652.0   
3  1.311448e+19   4/10/2019 0:31     486.0   4/25/2019 7:36   171.0   
4  1.311448e+19  4/12/2019 20:10     565.0  4/25/2019 10:40   954.0   

   Dur. (ms)          IMSI  MSISDN/Number          IMEI  \
0  1823652.0  2.082014e+14   3.366496e+10  3.552121e+13   
1  1365104.0  2.082019e+14   3.368185e+10  3.579401e+13   
2  1361762.0  2.082003e+14   3.376063e+10  3.528151e+13   
3  1321509.0  2.082014e+14   3.375034e+10  3.535661e+13   
4  1089009.0  2.082014e+14   3.369980e+10  3.540701e+13   

      Last Location Name  ...  Youtube DL (Bytes)  Youtube UL (Bytes)  \
0  9.16456699548519E+015  ...          15854611.0           2501332.0   
1                L77566A  ...          20247395.0          19111729.0   

In [2]:
network_parameteres=[
    'TCP DL Retrans. Vol (Bytes)',
    'TCP UL Retrans. Vol (Bytes)',
    'Avg RTT DL (ms)',
    'Avg RTT UL (ms)',
    'Avg Bearer TP DL (kbps)',
    'Avg Bearer TP UL (kbps)']
#preprocess numerical columns
from utils import DataProcessor
processor=DataProcessor()
df[network_parameteres]=processor.preprocess_numerical_data(df[network_parameteres]) 
#handle outliers
df[network_parameteres]=processor.remove_outliers_zscore(df[network_parameteres])


C:\Users\Ourba\Desktop\10Academy\10ac_week1_telecom_analysis\src\utils.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].fillna(df[col].mean())


In [41]:
# print(df[network_parameteres].describe())

 # Calculating the average TCP retransmission volume
df['Average TCP Retrans'] = df[['TCP DL Retrans. Vol (Bytes)', 'TCP UL Retrans. Vol (Bytes)']].sum().mean()
# # Displaying the average TCP retransmission volume for each user

print(df['Average TCP Retrans'])

0         1.127087e+12
1         1.127087e+12
2         1.127087e+12
3         1.127087e+12
4         1.127087e+12
              ...     
149996    1.127087e+12
149997    1.127087e+12
149998    1.127087e+12
149999    1.127087e+12
150000    1.127087e+12
Name: Average TCP Retrans, Length: 150001, dtype: float64


In [44]:
 # Calculating the average Rtt
df['Average RTT']= df[['Avg RTT DL (ms)','Avg RTT UL (ms)',]].sum().mean()
print(df['Average RTT'])

0         7.003717e+06
1         7.003717e+06
2         7.003717e+06
3         7.003717e+06
4         7.003717e+06
              ...     
149996    7.003717e+06
149997    7.003717e+06
149998    7.003717e+06
149999    7.003717e+06
150000    7.003717e+06
Name: Average RTT, Length: 150001, dtype: float64


In [45]:
 # Calculating the average tp
df['Average tp']= df[[ 'Avg Bearer TP DL (kbps)','Avg Bearer TP UL (kbps)',]].sum().mean()
print(df['Average tp'])

0         8.050943e+08
1         8.050943e+08
2         8.050943e+08
3         8.050943e+08
4         8.050943e+08
              ...     
149996    8.050943e+08
149997    8.050943e+08
149998    8.050943e+08
149999    8.050943e+08
150000    8.050943e+08
Name: Average tp, Length: 150001, dtype: float64


In [53]:
# Group by 'MSISDN/Number' and select specific columns for aggregation
aggs = df.groupby('MSISDN/Number')[['Average TCP Retrans', 'Average RTT', 'Average tp']].mean().reset_index()

print(aggs)


        MSISDN/Number  Average TCP Retrans   Average RTT    Average tp
0        3.360100e+10         1.127087e+12  7.003717e+06  8.050943e+08
1        3.360100e+10         1.127087e+12  7.003717e+06  8.050943e+08
2        3.360100e+10         1.127087e+12  7.003717e+06  8.050943e+08
3        3.360101e+10         1.127087e+12  7.003717e+06  8.050943e+08
4        3.360101e+10         1.127087e+12  7.003717e+06  8.050943e+08
...               ...                  ...           ...           ...
106851   3.379000e+10         1.127087e+12  7.003717e+06  8.050943e+08
106852   3.379000e+10         1.127087e+12  7.003717e+06  8.050943e+08
106853   3.197021e+12         1.127087e+12  7.003717e+06  8.050943e+08
106854   3.370000e+14         1.127087e+12  7.003717e+06  8.050943e+08
106855   8.823971e+14         1.127087e+12  7.003717e+06  8.050943e+08

[106856 rows x 4 columns]
